In [ ]:
import numpy as np
import tensorflow as tf
import scipy.io as sio
from sklearn.utils import resample, shuffle

In [ ]:
# the data should be organized as [date, open, close, low, high, volume]
data = sio.loadmat('../daily_data.mat')['data']

In [ ]:
data_len = data.shape[0]

# test data length
train_len = round(3 * data_len / 4)

x_train = data[0:train_len, :, :]
# start with just predicting the opening value the next day
y_train = data[1:train_len + 1, :, 1]

x_test = data[train_len + 1:-1, :, :]
# start with just predicting the opening value the next day
y_test = data[train_len + 2:, :, 1]

# let's start by just flattening the data
x_train = np.reshape(x_train, (train_len, -1))
x_test = np.reshape(x_test, (len(y_test), -1))

In [ ]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

In [ ]:
# def huber_loss(labels, predictions, delta=1.0):
#     residual = tf.abs(predictions - labels)
#     condition = tf.less(residual, delta)
#     small_res = 0.5 * tf.square(residual)
#     large_res = delta * residual - 0.5 * tf.square(delta)
#     return tf.select(condition, small_res, large_res)
def huber_loss(y_true, y_pred, max_grad=1.):
    """Calculates the huber loss.

    Parameters
    ----------
    y_true: np.array, tf.Tensor
      Target value.
    y_pred: np.array, tf.Tensor
      Predicted value.
    max_grad: float, optional
      Positive floating point value. Represents the maximum possible
      gradient magnitude.

    Returns
    -------
    tf.Tensor
      The huber loss.
    """
    err = tf.abs(y_true - y_pred, name='abs')
    mg = tf.constant(max_grad, name='max_grad')
    lin = mg*(err-.5*mg)
    quad=.5*err*err
    return tf.where(err < mg, quad, lin)

In [ ]:
input_len = x_train.shape[1]
output_len = y_train.shape[1]
hidden_len = round((input_len-output_len)/2.0)

# Create the model
# x = tf.placeholder(tf.float32, [None, input_len])
# W = tf.Variable(tf.zeros([input_len, output_len]))
# b = tf.Variable(tf.zeros([output_len]))
# y = tf.matmul(x, W) + b
x = tf.placeholder(tf.float32, [None, input_len])
W = weight_variable([input_len, hidden_len])
b = bias_variable([hidden_len])
x2 = tf.tanh(tf.matmul(x, W) + b)
# x2 = tf.nn.relu(tf.matmul(x, W) + b)

# add dropout
keep_prob = tf.placeholder(tf.float32)
x2_drop = tf.nn.dropout(x2, keep_prob)

W2 = weight_variable([hidden_len, output_len])
b2 = bias_variable([output_len])
y = tf.matmul(x2_drop, W2) + b2

# Define loss and optimizer
y_ = tf.placeholder(tf.float32, [None, output_len])

# The raw formulation of cross-entropy,
#
#   tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(tf.nn.softmax(y)),
#                                 reduction_indices=[1]))
#
# can be numerically unstable.
#
# So here we use tf.nn.softmax_cross_entropy_with_logits on the raw
# outputs of 'y', and then average across the batch.
# loss = tf.reduce_mean(
#   tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y))
# loss = tf.losses.mean_squared_error(labels=y_, predictions=y)
# loss = tf.reduce_mean(tf.square(y-y_))
loss = huber_loss(y_, y)
accuracy = tf.reduce_mean(tf.squared_difference(y, y_))
train_step = tf.train.AdamOptimizer(1e-2).minimize(loss)

sess = tf.InteractiveSession()
tf.global_variables_initializer().run()
# Train
for i in range(4000):
    batch_xs, batch_ys = resample(x_train, y_train, n_samples=100, random_state=0)
#     sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys})
    train_step.run(feed_dict={x: batch_xs, y_: batch_ys, keep_prob: 0.9})
    if i%100 == 0:
        train_accuracy = accuracy.eval(feed_dict={
            x:batch_xs, y_: batch_ys, keep_prob: 1.0})
        print("step %d, training accuracy %g"%(i, train_accuracy))
    

In [ ]:
print("test accuracy %g"%accuracy.eval(feed_dict={
x: x_test, y_: y_test, keep_prob: 1.0}))

In [ ]:
accuracy.eval(feed_dict={
x: x_test[0:1,:], y_: y_test[0:1,:], keep_prob: 1.0})

In [ ]:
y_test[0,:]

In [ ]:
sess.run(y, {x: x_test[0:1,:], keep_prob: 1.0})

In [ ]:
data.shape